#### 필요 라이브러리

In [ ]:
# 데이터 처리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import json
import itertools
import cv2
from torch.utils.data import DataLoader

# ViT
from timm import create_model, list_models
from types import SimpleNamespace

# GPT
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, get_linear_schedule_with_warmup

# 데이터 증강
import albumentations as A
from albumentations.pytorch import ToTensorV2 # 데이터 -> PyTorch 텐서로 변환
from PIL import Image # 이미지 처리
from pathlib import Path # 파일 경로 관리
from sklearn.model_selection import train_test_split
from torch.cuda.amp import GradScaler, autocast
from tqdm.auto import tqdm # 진행 상태 표시
import gc

# 최종 예측
import subprocess
import matplotlib.pyplot as plt
from IPython.display import display, Video

### Predictions

In [ ]:
model_path = '/content/drive/MyDrive/Trained_Model'

# best 모델 불러오기
trainer.load_best_model(model_path)

1. 분석하고 싶은 영상을 가져오기
2. 프레임으로 나누는 작업 진행
3. 결과 프레임들을 특정 경로(드라이브)에 저장
4. 프레임들 불러와서 new_frames_path 에 넣어주도록 generate_caption 함수를 조정했습니당

정리) 최종 프레임들 경로가 필요합니다

In [ ]:
# 경로들 정리

## 최종 모델 저장, 불러올 때 - /content/drive/MyDrive/Trained_Model
## 테스트용 비디오 불러올 때 - /content/drive/MyDrive/Testing_Videos => "testing_dir"
## 프레임 저장은 Testing_Videos 안에 폴더 만들어서 저장 (영상 이름으로 폴더 만들어서)

#### Framing & Predicting

In [ ]:
### 프레임 추출 + 캡션 생성 - 준희 추가

### 영상에서 프레임 추출
# video : 비디오 경로 / frames_dir : 프레임 추출해서 저장할 경로 / fps : 초당 추출할 프레임 수

def extract_frames(video, frames_dir, fps):
  os.makedirs(frames_dir, exist_ok = True)
  command = f"ffmpeg -i \"{video}\" -vf fps={fps} \"{frames_dir}/frame_%4d.jpg\""
  result = subprocess.run(command, shell=True, capture_output=True, text=True)

  # 에러 발생시를 위한..
  if result.returncode != 0:
    print(f"Error: {result.stderr}")

  return sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith(".jpg")])

### 영상 캡션 생성
# frames_path : 프레임들 저장된 폴더
def predicting(frames_path, transform):
    frames = os.listdir(frames_path)
    new_frames_path = [os.path.join(frames_dir, frame) for frame in frames]
    new_caption = trainer.generate_caption(new_frames_path, transform, temperature = 1.0, deterministic = False)
    return new_caption

In [ ]:
### 최종 함수
# video_name : 진짜 비디오 이름 / testing_dir : 영상 저장된 경로

def framing_and_predicting(video_name, testing_dir, fps):
    # 경로 지정
    video_dir = os.path.join(testing_dir, f"{video_name}.mp4")
    frames_dir = os.path.join(testing_dir, f"Frames/{video_name}")

    # 프레이밍, 저장
    frames = extract_frames(video_dir, frames_dir, fps = fps)
    print(f"Extracted {len(frames)} frames.")

    new_caption = predicting(frames_dir, transform)
    return new_caption

In [ ]:
# 일단 고양이 영상 하나에 테스팅
testing_dir = "/content/drive/MyDrive/Testing_Videos/Videos" # 영상이 저장된 경로

framing_and_predicting("이비 와작", testing_dir, fps = 2)

#### Framing & Predicting & Visualizing the Video

In [ ]:
def framing_and_predicting_and_visualizing(video_name, testing_dir, fps):
    # 경로 지정
    video_dir = os.path.join(testing_dir, f"{video_name}.mp4")
    frames_dir = os.path.join(testing_dir, f"Frames/{video_name}")

    # 프레이밍, 저장
    frames = extract_frames(video_dir, frames_dir, fps = fps)

    #########################################################
    # new_caption = predicting(frames_dir, transform)
    new_caption = "a cute fluffy kitty"

    # 비디오 표시
    display(Video(video_dir, embed=True, width=640, height=480))

    # 제목 텍스트 추가
    cap = cv2.VideoCapture(video_dir)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 영상이랑 가로 길이 맞췄으면 좋겠어서....

    plt.figure(figsize=(width / 90, 0.2), facecolor = 'black') # inch 기준은 96이라는데.. 흠
    plt.text(0.5, 0.5, f"^._.^ {new_caption} ^._.^", ha = 'center', va = 'center', color = 'white')
    plt.axis('off')
    plt.show()

In [ ]:
### 캡션 생성하고 비디오 재생하기
testing_dir = "/content/drive/MyDrive/Testing_Videos/Videos" # 영상이 저장된 경로

framing_and_predicting_and_visualizing("이비 와작", testing_dir, fps = 2 )